https://keon.github.io/deep-q-learning/

In [1]:
import numpy as np
import gym
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from collections import deque
import random

In [2]:
env = gym.make('CartPole-v1')

In [41]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.learning_rate = 0.001
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state[np.newaxis, :], verbose=0)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        """        
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0]))
            target_f = self.model.predict(state, verbose=0)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        """
        states = []
        actions = []
        rewards = []
        next_states = []
        dones = []
        for state, action, reward, next_state, done in minibatch:
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            next_states.append(next_state)
            dones.append(done)
        target_f = self.model.predict(np.array(states).reshape(32,4,1), verbose=0)
        target_next = self.model.predict(np.array(next_states).reshape(32,4,1), verbose=0)
        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(target_next[i][0]))
            target_f[i][action] = target
        self.model.fit(np.array(states).reshape(32,4,1), target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def learn(self, num_episodes):
        for e in range(num_episodes):
            state, info = env.reset()
            state = state[np.newaxis, :]

            for time_t in range(500):
                action = self.act(state)
                next_state, reward, done, trunctaed, info = env.step(action)
                next_state = next_state[np.newaxis, :]
                
                self.memorize(state, action, reward, next_state, done)

                state = next_state

                if done:
                    print("episode: {}/{}, score: {}, e: {:.2}".format(e, num_episodes, time_t, self.epsilon))
                    break
                if len(self.memory) > 32:
                    self.replay(32)

In [42]:
agent = DQNAgent(4, 2)
agent.learn(1000)

episode: 0/1000, score: 15, e: 1.0
episode: 1/1000, score: 12, e: 1.0
episode: 2/1000, score: 12, e: 0.96
episode: 3/1000, score: 18, e: 0.87
episode: 4/1000, score: 14, e: 0.81
episode: 5/1000, score: 9, e: 0.78
episode: 6/1000, score: 11, e: 0.74
episode: 7/1000, score: 17, e: 0.68
episode: 8/1000, score: 8, e: 0.65
episode: 9/1000, score: 12, e: 0.61
episode: 10/1000, score: 12, e: 0.58
episode: 11/1000, score: 9, e: 0.55
episode: 12/1000, score: 8, e: 0.53
episode: 13/1000, score: 13, e: 0.5
episode: 14/1000, score: 22, e: 0.44
episode: 15/1000, score: 14, e: 0.41
episode: 16/1000, score: 11, e: 0.39
episode: 17/1000, score: 14, e: 0.37
episode: 18/1000, score: 8, e: 0.35
episode: 19/1000, score: 9, e: 0.34
episode: 20/1000, score: 12, e: 0.32
episode: 21/1000, score: 9, e: 0.3
episode: 22/1000, score: 10, e: 0.29
episode: 23/1000, score: 12, e: 0.27
episode: 24/1000, score: 7, e: 0.26
episode: 25/1000, score: 12, e: 0.25
episode: 26/1000, score: 9, e: 0.23
episode: 27/1000, score: